# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846816451630                   -0.70    5.0
  2   -7.852313647174       -2.26       -1.53    1.0
  3   -7.852614479974       -3.52       -2.55    1.5
  4   -7.852645909342       -4.50       -2.87    2.5
  5   -7.852646476663       -6.25       -3.15    1.0
  6   -7.852646679059       -6.69       -3.98    1.2
  7   -7.852646686302       -8.14       -5.12    1.5
  8   -7.852646686718       -9.38       -5.31    2.2
  9   -7.852646686728      -10.99       -5.78    1.0
 10   -7.852646686730      -11.89       -6.27    1.2


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846812037536                   -0.70           4.8
  2   -7.852526620861       -2.24       -1.63   0.80    2.0
  3   -7.852637136510       -3.96       -2.70   0.80    1.0
  4   -7.852646391807       -5.03       -3.29   0.80    2.2
  5   -7.852646681776       -6.54       -4.10   0.80    1.8
  6   -7.852646686394       -8.34       -4.71   0.80    1.2
  7   -7.852646686721       -9.49       -5.70   0.80    1.8
  8   -7.852646686730      -11.07       -6.55   0.80    1.8


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.413963e+01     3.475827e+00
 * time: 0.4409949779510498
     1     1.346364e+00     1.836946e+00
 * time: 0.6494460105895996
     2    -1.204895e+00     2.206977e+00
 * time: 0.6772518157958984
     3    -3.683869e+00     1.805504e+00
 * time: 0.7166659832000732
     4    -4.960450e+00     1.716476e+00
 * time: 0.7557389736175537
     5    -6.740076e+00     9.626648e-01
 * time: 0.7951200008392334
     6    -6.805235e+00     1.314835e+00
 * time: 0.8221139907836914
     7    -7.480720e+00     8.695507e-01
 * time: 0.8495128154754639
     8    -7.575608e+00     1.089775e+00
 * time: 0.8769748210906982
     9    -7.743979e+00     8.096489e-01
 * time: 0.90451979637146
    10    -7.784605e+00     7.020583e-01
 * time: 0.9312939643859863
    11    -7.811131e+00     5.367578e-01
 * time: 0.9581148624420166
    12    -7.830093e+00     3.325562e-01
 * time: 0.9847619533538818
    13    -7.841716e+00     8.259944e-02
 * time: 1.091745853424

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846762994721                   -0.70    4.5


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852645850951                   -1.64
  2   -7.852646686730       -6.08       -3.71
  3   -7.852646686730      -13.14       -7.19


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.802922845733138e-7
|ρ_newton - ρ_scfv| = 1.5435950834374442e-7
|ρ_newton - ρ_dm|   = 9.594311464375796e-10
